### Plot 1

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

#Define and plot optimal proposal
ground_truth = 0.03283152373679992
def joint_density(x):
    return sp.stats.gamma.pdf(x, a=5, loc=0, scale=4) * sp.stats.norm.pdf(5, loc=x, scale=1)
def optimal_proposal(x):
    return np.abs(np.minimum(15000, np.maximum(0, 50*(x-8)**5)) - ground_truth) * joint_density(x)
def target_joint_density(x):
    return np.minimum(15000, np.maximum(0, 50*(x-8)**5)) * joint_density(x)
def alternative_proposal(x):
    return np.sqrt(target_joint_density(x)*joint_density(x))

#plot optimal proposal
x = np.linspace(0, 12, 1000)
plt.figure(figsize=(10, 5))
plt.plot(x,8*optimal_proposal(x), label='Optimal TARIS proposal')
plt.plot(x,joint_density(x), label='p(x,y)')
plt.plot(x,20*target_joint_density(x), label='f(x)p(x,y)')
plt.xlabel('x')
plt.xlim(2, 12)
plt.legend()


### Plot 2

In [ ]:
### using scikit learn
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy.stats import norm

#Define and plot optimal proposal
ground_truth = 0.03283152373679992
def joint_density(x):
    return sp.stats.gamma.pdf(x, a=5, loc=0, scale=4) * sp.stats.norm.pdf(5, loc=x, scale=1)
def optimal_proposal(x):
    return np.abs(np.minimum(15000, np.maximum(0, 50*(x-8)**5)) - ground_truth) * joint_density(x)

# Generate synthetic data
np.random.seed(0)
samples1 = np.random.normal(5.75, 1.5, 1000)
samples2 = np.random.normal(9.2, 0.5, 1000)
combined_samples = np.concatenate((samples1, samples2)).reshape(-1, 1)

# Fit a Gaussian Mixture Model with 2 components
gmm = GaussianMixture(n_components=2)
gmm.fit(combined_samples)

# Generate samples from the fitted GMM
generated_samples = gmm.sample(2000)[0]

#evaluate density of the generated samples
log_density_of_generated_samples = gmm.score_samples(generated_samples.reshape(-1, 1))
density_of_generated_samples = np.exp(log_density_of_generated_samples)

# Plot the histogram of the samples
plt.hist(generated_samples, bins=50, density=True, alpha=0.6)

# Plot the individual Gaussian components of the GMM
x = np.linspace(2, 12, 1000)
plt.plot(x, norm.pdf(x, gmm.means_[0][0], np.sqrt(gmm.covariances_[0][0][0])), label='Component 1', color='red')
plt.plot(x, norm.pdf(x, gmm.means_[1][0], np.sqrt(gmm.covariances_[1][0][0])), label='Component 2', color='blue')
plt.plot(x, 1900*optimal_proposal(x), label='Optimal TARIS proposal', color='green')

plt.xlabel("Value")
plt.ylabel("Density")
plt.title("Gaussian Mixture Model with Two Components")
plt.legend()


### Plot 3

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt
from TARIS import TARIS_GMM_PROPOSAL

result_100k = TARIS_GMM_PROPOSAL(100,101000)

#extract results from TARIS.ipynb file
r_plot_100k = result_100k['ratio_estimate_per_iteration']

#calculate averages
result_100k = r_plot_100k[5:]
averages_100k  = np.zeros(95)

for i in range(95):
    averages_100k[i] = np.mean(result_100k[:i+1])
    
averages_100k
np.var(averages_100k)

ground_truth = 0.03283152373679992

#plot experiment results and averages
plt.figure(figsize=(10,10))
plt.plot(r_plot_100k)
plt.plot(averages_100k)
plt.axhline(y=ground_truth, color='r', linestyle='--')
plt.legend(['TARIS','Averaged TARIS','Ground truth'])

### Plot 4

In [ ]:
from TARIS import TARIS_GMM_PROPOSAL, calculate_TARIS_ReMSE, compute_TABI

result_100 = TARIS_GMM_PROPOSAL(1000,1100)
result_1k = TARIS_GMM_PROPOSAL(1000,2000)
result_10k = TARIS_GMM_PROPOSAL(100,11000)

ReMSE_10k, ReMSE_1k, ReMSE_100, total_samples_1k, total_samples_10k, total_samples_100 \
    = calculate_TARIS_ReMSE(result_100, result_1k, result_10k)

#calculate ReMSE for TABI
total_samples_TABI = np.unique(np.concatenate((total_samples_100, total_samples_1k, total_samples_10k)))/2
total_samples_TABI = total_samples_TABI.astype(int)

results_TABI = {n: compute_TABI(n, n) for n in total_samples_TABI}

#extract results and ReMSE
Re_MSEs_TABI = [results_TABI[n]['ReMSE'] for n in total_samples_TABI]
ReMSE_TABI = np.array(Re_MSEs_TABI)

# plot the ReMSEs
plt.figure(figsize=(10, 6))
plt.plot(total_samples_1k, ReMSE_1k, label='ReMSE 1k')
plt.plot(total_samples_10k, ReMSE_10k, label='ReMSE 10k')
plt.plot(total_samples_100, ReMSE_100, label='ReMSE 100')
plt.plot(total_samples_TABI, Re_MSEs_TABI, label='ReMSE TABI')

plt.xscale('log')
plt.yscale('log')

plt.xlabel('Total Samples Used')
plt.ylabel('Relative Mean Squared Error (ReMSE)')
plt.title('ReMSE of different N valued functions against total samples used')
plt.legend()
plt.grid(True, which="both", ls="--", linewidth=0.5)
